In [1]:
import cfg

running price stream
running transaction stream
running account update polling


In [2]:
cfg.account

In [3]:
print(cfg.account.trades)

[]


In [4]:
cfg.account.balance

1608.9

In [7]:
cfg.instruments['EUR_USD']

{'name': 'EUR_USD',
 'type': 'CURRENCY',
 'displayName': 'EUR/USD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1.0',
 'maximumTrailingStopDistance': '1.0',
 'minimumTrailingStopDistance': '0.0005',
 'maximumPositionSize': '0.0',
 'maximumOrderUnits': '100000000.0',
 'marginRate': '0.02',
 'bid': 1.1312,
 'ask': 1.1317,
 'spread': 0.0005}